In [ ]:
pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data from the CSV file
loader = CSVLoader(file_path="2019.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)



#Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

# Converting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

DB_FAISS_PATH = "/content/vectorstore/db_faiss"
docsearch.save_local(DB_FAISS_PATH)


#docs = docsearch.similarity_search(query, k=3)
#print("Result", docs)

llm = CTransformers(model="/content/drive/MyDrive/models/llama-2-7b-chat.Q3_K_M.gguf",
                    model_type="llama",
                    max_new_tokens=512,
                    temperature=0.1)

qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())


# Load data into a pandas DataFrame for easier statistical analysis
df = pd.read_csv('2019.csv')

def calculate_statistics(selected_column):
    mean = np.mean(selected_column)
    median = np.median(selected_column)
    std_dev = np.std(selected_column)
    return mean, median, std_dev

while True:
    choice = input("Enter 'q' to ask a question, 's' to perform statistical analysis, or 'p' to plot a graph: ")

    if choice == 'q':
        chat_history = []
        query = input("Input Prompt: ")
        if query == 'exit':
            print('Exiting')
            sys.exit()
        if query == '':
            continue
        result = qa({"question": query, "chat_history": chat_history})
        print("Response: ", result['answer'])

    elif choice == 's':
        # Perform statistical analysis

        # Display available column names
        print("Available column names:")
        print(df.columns)

        column_name = input("Enter the column name you want to analyze: ")
        selected_column = df[column_name]  # Use column_name directly
        mean, median, std_dev = calculate_statistics(selected_column)
        print(f"Mean: {mean}")
        print(f"Median: {median}")
        print(f"Standard Deviation: {std_dev}")

    elif choice == 'p':
        # Plot a graph

        # Display available column names
        print("Available column names:")
        print(df.columns)

        column_name1 = input("Enter the column name for the x-axis: ")
        column_name2 = input("Enter the column name for the y-axis: ")

        # Check if both column names are valid
        if column_name1 in df.columns and column_name2 in df.columns:
            x_data = df[column_name1]
            y_data = df[column_name2]

            # Plot histogram if both columns are numeric
            if pd.api.types.is_numeric_dtype(x_data) and pd.api.types.is_numeric_dtype(y_data):
                plt.hist(x_data, bins='auto')
                plt.xlabel(column_name1)
                plt.ylabel('Frequency')
                plt.title('Histogram')
                plt.show()

            # Plot scatter plot if both columns are numeric
            elif pd.api.types.is_numeric_dtype(x_data) and pd.api.types.is_numeric_dtype(y_data):
                plt.scatter(x_data, y_data)
                plt.xlabel(column_name1)
                plt.ylabel(column_name2)
                plt.title('Scatter Plot')
                plt.show()

            # Plot line plot if both columns are numeric
            elif pd.api.types.is_numeric_dtype(x_data) and pd.api.types.is_numeric_dtype(y_data):
                plt.plot(x_data, y_data)
                plt.xlabel(column_name1)
                plt.ylabel(column_name2)
                plt.title('Line Plot')
                plt.show()

            else:
                print("At least one of the selected columns is not numeric. Please select numeric columns.")
        else:
            print("One or both of the column names are invalid.")

    else:
        print("Invalid choice. Please enter 'q' to ask a question, 's' to perform statistical analysis, or 'p' to plot a graph.")


Enter 'q' to ask a question, 's' to perform statistical analysis, or 'p' to plot a graph: q
Input Prompt: GDP of Taiwan


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response:   A person with a score of 6.446 would expect the GDP of Taiwan to be around $25,000 per capita.


KeyboardInterrupt: Interrupted by user